Assignment 6

In [122]:
import numpy as np

stuff=np.load("data.npz")
X_trn = stuff["X_trn"]
y_trn = stuff["y_trn"]
X_tst = stuff["X_tst"]
# no Y_tst !

In [123]:
# CSV Helper Function

# import numpy as np
# import csv

# def write_csv(y_pred, filename):
#     """Write a 1d numpy array to a Kaggle-compatible .csv file"""
#     with open(filename, 'w') as csv_file:
#         csv_writer = csv.writer(csv_file)
#         csv_writer.writerow(['Id', 'Category'])
#         for idx, y in enumerate(y_pred):
#             csv_writer.writerow([idx, y])


# data = np.load('data.npz')
# X_tst = data['X_tst']
# y_pred = np.random.randint(0, 3, size=len(X_tst)) # random predictions
# write_csv(y_pred, 'sample_predictions.csv')

In [124]:
# # Example plot

# from matplotlib import pyplot as plt

# def show(x):
#     img = x.reshape((3,29,29)).transpose(1,2,0)
#     plt.imshow(img)
#     plt.axis('off')
#     plt.draw()
#     plt.pause(0.01)

# show(X_trn[7])

Question 1

In [125]:
X = [1.0, 2.0, 3.0, 4.0, 5.0]
Y = [1, 1, 0, 0, 1]
splits = [0.5, 1.5, 2.5, 3.5, 4.5, 5.5]

# calculate parent entropy

parent_p1 = sum(Y) / len(Y)
parent_p0 = 1 - parent_p1
parent_entropy = (-parent_p1*np.log2(parent_p1)) + (- parent_p0*np.log2(parent_p0))

# calculate child entropy for each split
child_entropy = {.5: -1, 1.5: -1, 2.5: -1, 3.5: -1, 4.5: -1, 5.5: -1}

for split in splits:
    # split data
    left = []
    right = []
    for i in range(len(X)):
        if X[i] < split:
            left.append(Y[i])
        else:
            right.append(Y[i])

    # calculate I(p1) and I(p2) for both left and right sides (if empty, set to 0)
    left_p1 = 0
    if len(left) != 0:
        left_p1 = sum(left) / len(left)
    left_p0 = 1 - left_p1

    right_p1 = 0
    if len(right) != 0:
        right_p1 = sum(right) / len(right)
    right_p0 = 1 - right_p1

    if left_p1 == 0:
        left_entropy = -left_p0*np.log2(left_p0)
    elif left_p0 == 0:
        left_entropy = -left_p1*np.log2(left_p1)
    else:
        left_entropy = (-left_p1*np.log2(left_p1)) + (- left_p0*np.log2(left_p0))
    
    if right_p1 == 0:
        right_entropy = -right_p0*np.log2(right_p0)
    elif right_p0 == 0:
        right_entropy = -right_p1*np.log2(right_p1)
    else:
        right_entropy = (-right_p1*np.log2(right_p1)) + (- right_p0*np.log2(right_p0))
    
    # calculate child entropy and store
    child_entropy[split] = ((len(left))*left_entropy + (len(right))*right_entropy) / len(Y)

# calculate information gain and print
information_gain = {split: parent_entropy - child_entropy[split] for split in splits}

print("Split | Information Gain")
for split in splits:
    print(f" {split}  |  {information_gain[split]}")

Split | Information Gain
 0.5  |  0.0
 1.5  |  0.17095059445466854
 2.5  |  0.4199730940219748
 3.5  |  0.01997309402197489
 4.5  |  0.17095059445466854
 5.5  |  0.0


Question 2

In [126]:
# class ClassificationStump():
#     def __init__(self):
#         return

#     def fit(self, X_trn, y_trn):

       
#         # do stuff here
#         D = len(X_trn[0])
#         N = len(X_trn)
#         dim = -1
#         thresh = -1
#         min_error = np.inf
#         c_left = 0
#         c_right = 0

#         for i in range(D):
#             print(i)
#             Z = sorted(X_trn,key=lambda x: x[i])
#             for n in range(N-1):
#                 t = (Z[n][i] + Z[n+1][i])/2
#                 R1 = [n for n in range(N) if X_trn[n][i] <= t]
#                 R2 = [n for n in range(N) if X_trn[n][i] > t]
                
#                 # REGRESSION: c1 = np.mean([y_trn[n] for n in R1]) # Change for classification. Get the most common class in R1. Break ties by choosing the smaller class (does not matter how).
#                 R1_classes = np.bincount([y_trn[n] for n in R1])
#                 c1 = 0
#                 if len(R1_classes) > 0:
#                     c1 = np.argmax(R1_classes) # CHECK THAT YOU ARE ALLOWED TO USE THESE FUNCTIONS

#                 # REGRESSION: c2 = np.mean([y_trn[n] for n in R2]) # Change for classification. Get the most common class in R2. Break ties by choosing the smaller class (does not matter how).
#                 R2_classes = np.bincount([y_trn[n] for n in R2])
#                 c2 = 0
#                 if len(R2_classes) > 0:
#                     c2 = np.argmax(R2_classes) # CHECK THAT YOU ARE ALLOWED TO USE THESE FUNCTIONS

#                 # REGRESSION: error = sum([(y_trn[n] - c1)**2 for n in R1]) + sum([(y_trn[n] - c2)**2 for n in R2]) # Change for classification. Use Classification error: 1/N SUM i=1 to N of misclassifications. i.e. average number of misclassifications.
#                 error = sum([1 for n in R1 if y_trn[n] != c1]) + sum([1 for n in R2 if y_trn[n] != c2])

#                 if error < min_error:
#                     min_error = error
#                     dim = i
#                     c_left = c1
#                     c_right = c2
#                     thresh = t

#         self.model = (dim, thresh, c_left, c_right)
#         return min_error

#     def predict(self, X_val, y_val=None):
#         assert hasattr(self, "model"), "No fitted model!"
#         # do stuff here (use self.model for prediction)
#         y_pred = self.model.c_left if X_val[self.model.dim] <= self.model.thresh else self.model.c_right
#         return y_pred



In [150]:

class ClassificationStump():
    def __init__(self):
        return

    def fit(self, X_trn, y_trn):
        
        # do stuff here
        D = len(X_trn[0])
        N = len(X_trn)
        dim = -1
        thresh = -1
        min_error = np.inf
        c_left = 0
        c_right = 0

        for i in range(D):

            sorted_indices = np.argsort(X_trn[:, i])
            Z = X_trn[sorted_indices]
            y_sorted = y_trn[sorted_indices]

            # Initialize class counts for R1 and R2
            # R1_count = np.zeros(np.max(y_trn) + 1, dtype=int)
            # R2_count = np.bincount(y_sorted)

            for n in range(N-1):
                t = (Z[n][i] + Z[n+1][i])/2

                R1 = y_sorted[:n+1] # x <= t
                R2 = y_sorted[n+1:] # x > t

                R1_count = np.bincount(R1)
                R2_count = np.bincount(R2)
                
                c1 = np.argmax(R1_count)
                c2 = np.argmax(R2_count)

                # gini_left = (R1_count[c1] / len(R1)) * ((len(R1) - R1_count[c1]) / len(R1))
                # gini_right = (R2_count[c2] / len(R2)) * ((len(R2) - R2_count[c2]) / len(R2))
                p1 = R1_count[c1] / len(R1)
                
                p2 = R2_count[c2] / len(R2)
                
                gini_left = p1 * (1-p1)   # sum pi(1-pi) for each class in R1: p1 * (1-p1) + p2 * (1-p2)
                gini_right = p2 * (1-p2)  # sum pi(1-pi) for each class in R2: p1 * (1-p1) + p2 * (1-p2)
                gini_total = (len(R1) / N) * gini_left + (len(R2) / N) * gini_right # OH: NOT WORKING PROPERLY (always predicting 0? - although that could make sense since we have more than 2 classes)
                #error = ((len(R1) - R1_count[c1]) + (len(R2) - R2_count[c2])) / N # Classification error - should be ... Gini? Cross-entropy? Something else?
                error = gini_total
                if error < min_error:
                    min_error = error
                    dim = i
                    c_left = c1
                    c_right = c2
                    thresh = t

        self.model = (dim, thresh, c_left, c_right)
        self.model = {'dim': dim, 'thresh': thresh, 'c_left': c_left, 'c_right': c_right}
        return min_error

    def predict(self, X_val, y_val=None):
        assert hasattr(self, "model"), "No fitted model!"
        # do stuff here (use self.model for prediction)
        y_pred = []
        for x_val in X_val:
            y_pred.append(self.model['c_left'] if x_val[self.model['dim']] <= self.model['thresh'] else self.model['c_right'])
        
        return y_pred



In [151]:

data = np.load('data.npz')
X_trn = data['X_trn']
y_trn = data['y_trn']
X_tst = data['X_tst']

clf = ClassificationStump()

# reshape from (N, 3, 841) to (N, 2523)

# X_trn = X_trn.reshape((len(X_trn), -1)) -> shouldnt change answer
X_trn = X_trn.reshape((6000, 3 * 29 * 29))
y_trn = y_trn.reshape((len(y_trn), )) # .reshape(len(y_trn), 1) did not work but worked in 10?

clf.fit(X_trn, y_trn)
y_pred = clf.predict(X_trn)
print(y_pred)
correct_pred = sum(y_pred == y_trn)
e = 1 - correct_pred / len(y_pred)
print("Training classification error: ", e)



[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [152]:
stuff = np.load("data.npz")
X_trn = stuff["X_trn"]
X_trn = X_trn.reshape((6000, 3 * 29 *29))

y_pred = clf.predict(X_trn)
print(y_trn)
print(y_pred)

[1 3 1 ... 3 1 2]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

Question 3

In [130]:
# train classifcation trees
from sklearn.tree import DecisionTreeClassifier

data = np.load('data.npz')
X_trn = data['X_trn']
y_trn = data['y_trn']

X_trn = X_trn.reshape((len(X_trn), -1))

max_depths = [1,3,6,9,12,14]
# return a 6x1 table of classification errors
print(" DEPTH | TRAINING ERROR")
for max_depth in max_depths:
    clf = DecisionTreeClassifier(max_depth=max_depth)
    clf.fit(X_trn, y_trn)
    e = 1 - clf.score(X_trn, y_trn) # OH: This or use clf.predict(X_trn) and compare to y_trn?
    print(f"  {max_depth}  | {e}")

 DEPTH | TRAINING ERROR
  1  | 0.6433333333333333
  3  | 0.5515
  6  | 0.4033333333333333
  9  | 0.2496666666666667
  12  | 0.1293333333333333
  14  | 0.07150000000000001


Question 4

Question 5

Question 6

In [131]:
# linear classifier model with logistic loss and ridge regularization only using sklearn.linear and decision_function() method

from sklearn.linear_model import LogisticRegression

data = np.load('data.npz')
X_trn = data['X_trn']
y_trn = data['y_trn']
X_tst = data['X_tst']

# X_trn = X_trn.reshape((len(X_trn), -1)) -> Did not change answer
X_trn = X_trn.reshape((6000, 3*29*29))  

# y_trn = y_trn.reshape((len(y_trn), -1)) -> Did not change answer
# # convert y_trn to 1d array
# y_trn = y_trn.transpose()[0]

lambda_vals = [0.1, 1, 10, 100, 1000]

# report training classification error and logistic loss for each lambda value
print(" LAMBDA | TRAINING ERROR | LOGISTIC LOSS")
for lambda_val in lambda_vals:
    clf = LogisticRegression(penalty='l2', C=1/lambda_val, multi_class='auto', max_iter=10000, tol=.001)
    clf.fit(X_trn, y_trn)
    confidence_scores = clf.decision_function(X_trn) # get confidence score per class per sample -> get logistic loss per class per sample, get classification error, get average logistic loss (?), 
    
    # multi logistic loss = L(y, f(x)) = -f_y(x) +  log(sum(exp(f_v(x)))). 
    L = -confidence_scores[y_trn] + np.log(sum(np.exp(confidence_scores))) # get logistic loss per class per sample - ?
    y_pred = [np.argmin(loss_scores) for loss_scores in L] # get the class with lowest loss - ?
    correct_preds = np.sum(y_pred == y_trn)
    logistic_loss = np.mean(L) # get average logistic loss - ?
    e = 1 - (correct_preds / len(y_trn))
    print(f"  {lambda_val}  | {e} | {logistic_loss}")


 LAMBDA | TRAINING ERROR | LOGISTIC LOSS
  0.1  | 1.0 | 19.098087119986353
  1  | 1.0 | 14.281281254108777
  10  | 1.0 | 12.244757863004764
  100  | 0.75 | 11.09240015821819
  1000  | 0.75 | 9.927447769712384


Question 7

In [132]:
# K nearest neighbors classifier
from sklearn.neighbors import KNeighborsClassifier

data = np.load('data.npz')
X_trn = data['X_trn']
y_trn = data['y_trn']
X_tst = data['X_tst']

X_trn = X_trn.reshape((len(X_trn), -1))

k_neighbors = [1, 3, 5, 7, 9, 11]

print(" K | TRAINING ERROR")
for k in k_neighbors:
    clf = KNeighborsClassifier(n_neighbors=k)
    clf.fit(X_trn, y_trn)
    e = 1 - clf.score(X_trn, y_trn)
    print(f" {k} | {e}")

 K | TRAINING ERROR
 1 | 0.0
 3 | 0.29200000000000004
 5 | 0.31633333333333336
 7 | 0.3478333333333333
 9 | 0.3626666666666667
 11 | 0.3663333333333333


Question 8

PART 2 - Cross Validation

Question 9

In [155]:
from sklearn.model_selection import KFold

class Classifier():
    def __init__(self, model):
        self.model = model
        return

    def fit(self, X_trn, y_trn):
        self.model.fit(X_trn, y_trn)
        # self.model is stored
        return

    def predict(self, X_val, y_val=None):
        # self.model is used
        y_pred = self.model.predict(X_val)
        return y_pred


def cross_validation(classifier, X_trn, y_trn, n_folds=5):
    # do stuff here

    kf = KFold(n_splits=n_folds)
    outputs = []

    for train_index, test_index in kf.split(X_trn):
        
        x_train, x_test = X_trn[train_index], X_trn[test_index]
        y_train, y_test = y_trn[train_index], y_trn[test_index]
        
        # convert test to 1d array
        # y_train = y_train.transpose()[0] # NEEDED for Q11 but not for Q10?
        # y_test = y_test.transpose()[0]

        classifier.fit(x_train, y_train)
        y_pred = classifier.predict(x_test)
        correct_preds = np.sum(y_pred == y_test)
        e = 1 - (correct_preds / len(y_test))
        
        outputs.append((classifier, e))
    
    # Return the paired (model and error) for all folds
    return outputs # [(model1, error1), (model2, error2), ..., (modelK, errorK)]
    



Question 10

In [140]:
# Usage for the cross_validation function:
from sklearn.tree import DecisionTreeClassifier

data = np.load('data.npz')
X_trn = data['X_trn']
y_trn = data['y_trn']

# X_trn = X_trn.reshape((len(X_trn), -1))
X_trn = X_trn.reshape((6000, 3 * 29 * 29))
# y_trn = y_trn.reshape((len(y_trn), 1)) # -> MUST DO IN ORDER TO PRODUCE ANSWER EVEN THOUGH SHAPE SEEMS TO BE THE SAME (because of transpose..)
# y_trn = y_trn.reshape((1200, 1))
# y_trn = y_trn.reshape((len(y_trn), -1))
# print(y_trn.shape)

print(" MAX_DEPTH | FOLD_1_ERROR | FOLD_2_ERROR | FOLD_3_ERROR | FOLD_4_ERROR | FOLD_5_ERROR | AVG_ERROR")
for max_depth in [1, 3, 6, 9, 12, 14]:
    classifier = Classifier(DecisionTreeClassifier(max_depth=max_depth))
    N = 5
    outputs = cross_validation(classifier, 
                           X_trn, 
                           y_trn, 
                           n_folds=N)
    print(f" {max_depth} | ", end="")
    for out_ in outputs:
        print(f"{out_[1]} | ", end="")
    avg_error = np.mean([out_[1] for out_ in outputs])
    print(f"{avg_error}")
    


 MAX_DEPTH | FOLD_1_ERROR | FOLD_2_ERROR | FOLD_3_ERROR | FOLD_4_ERROR | FOLD_5_ERROR | AVG_ERROR
 1 | 0.6541666666666667 | 0.6616666666666666 | 0.6358333333333333 | 0.6375 | 0.63 | 0.6438333333333333
 3 | 0.5275000000000001 | 0.5691666666666666 | 0.565 | 0.5475 | 0.54 | 0.5498333333333333
 6 | 0.4866666666666667 | 0.4933333333333333 | 0.4966666666666667 | 0.4833333333333333 | 0.4816666666666667 | 0.4883333333333333
 9 | 0.4666666666666667 | 0.4966666666666667 | 0.47583333333333333 | 0.4625 | 0.4591666666666666 | 0.4721666666666667
 12 | 0.4708333333333333 | 0.47916666666666663 | 0.47916666666666663 | 0.47 | 0.47 | 0.47383333333333333
 14 | 0.48583333333333334 | 0.5008333333333334 | 0.4708333333333333 | 0.4783333333333334 | 0.4683333333333334 | 0.48083333333333333


Question 11

In [156]:
from sklearn.linear_model import LogisticRegression

data = np.load('data.npz')
X_trn = data['X_trn']
y_trn = data['y_trn']
#print(y_trn.shape) -> prints (6000,)
X_trn = X_trn.reshape((len(X_trn), -1))
# X_trn = X_trn.reshape((6000, 3 * 29 * 29))
# y_trn = y_trn.reshape((len(y_trn), -1)) # OH: CHECK THIS, changes the answer -> needed to work
# y_trn = y_trn.reshape((1200, 1))

print(" LAMBDA | FOLD_1_ERROR | FOLD_2_ERROR | FOLD_3_ERROR | FOLD_4_ERROR | FOLD_5_ERROR | AVG_ERROR")
for lambda_val in [0.1, 1, 10, 100, 1000]:
    classifier = Classifier(LogisticRegression(penalty='l2', C=1/lambda_val, multi_class='auto', max_iter=10000, tol=.001))
    N = 5
    outputs = cross_validation(classifier, 
                           X_trn, 
                           y_trn, 
                           n_folds=N)
    print(f" {lambda_val} | ", end="")
    for out_ in outputs:
        print(f"{out_[1]} | ", end="")
    avg_error = np.mean([out_[1] for out_ in outputs]) 
    print(f"{avg_error}")

# OH: SHOULDNT ERRORS INCREASE WITH LAMBDA? CHECK THIS

 LAMBDA | FOLD_1_ERROR | FOLD_2_ERROR | FOLD_3_ERROR | FOLD_4_ERROR | FOLD_5_ERROR | AVG_ERROR
 0.1 | 0.45833333333333337 | 0.4833333333333333 | 0.48250000000000004 | 0.47250000000000003 | 0.48083333333333333 | 0.4755000000000001
 1 | 0.42333333333333334 | 0.4275 | 0.43500000000000005 | 0.4291666666666667 | 0.4125 | 0.4255
 10 | 0.3766666666666667 | 0.4075 | 0.39249999999999996 | 0.3966666666666666 | 0.3533333333333334 | 0.38533333333333336
 100 | 0.3433333333333334 | 0.37916666666666665 | 0.3483333333333334 | 0.3616666666666667 | 0.3275 | 0.35200000000000004
 1000 | 0.35083333333333333 | 0.37416666666666665 | 0.3516666666666667 | 0.35750000000000004 | 0.33999999999999997 | 0.3548333333333333
